# Introduction to Medical Image Registration using Deep Neural Networks with DeepReg

### Nina Montana Brown, Yunguan Fu, Shaheer U. Saeed, Zac Baum, Adria Casamitjana, Alex Grimwood, Ester Bonmati, Tom Vercauteeren, Matt Clarkson, Yipeng Hu

### Who am I missing? Affiliations?

# Table of Contents - Update
1. [Set-up](#setup)
2. [Introduction to Registration](#IntroReg)
3. [Registration with Deep Learning](#DeepRegistrationIntro)
4. [Two classical registration examples](#classical-examples) 
5. [Medical Image registration using an adapted DeepReg Demo](#deep-example)
6. [References](#references)

# Objective
This tutorial introduces a new open-source project [DeepReg](https://github.com/DeepRegNet/DeepReg), which is designed to be a rookie-friendly package for researchers interested in image registration using deep learning. 

A previous MICCAI workshop [learn2reg](https://learn2reg.github.io/) provided an excelent example of novel algorithms and interesting approaches in this active research area, whilst this tutorial explores the strenghth of the simple, yet generalisable deisgn of DeepReg. 
- Explain basic concepts in medical image registration;
- Explore the links between the modern algorithms using neural networks and the classical iterative algorithms (which can also be readily implemented using DeepReg!);
- Perhaps more importantly, introduce the versatile capability of DeepReg, with diverse examples in real clinical challenges.

All of this will require minimum scripting and coding expereince with DeepReg, accomanying a set of well-written documentation and a growing number of demos using real, open-accesible clinical data. This tutorial will get you started with a number of examples by step-by-step instructions.

# Set-up <a name="setup"></a>
This tutorial depends on the package [DeepReg](https://github.com/DeepRegNet/DeepReg), which in turn has external dependencies which are managed by `pip`. 

To ensure the demo'd algorithms can run from this Google Colab notebook, you will need to mount your Google Drive and clone the repository into it. You can do this by running the following commands.

You can also follow along in a local copy of the repo - in this case, follow instructions for set up at [the quickstart guide](https://github.com/DeepRegNet/DeepReg/blob/master/docs/quick_start.md).

Training DNNs is computationally expensive. We have tested this demo with GPUs provided by Google through Google Colab. Training times have been roughly measured and indicated where appropriate. You can run this on CPU but we have not tested how long it would take.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')
! pwd


KeyboardInterrupt: ignored

Once the drive is mounted, ensure that you have GPU enabled for more efficient training.

To do this, go to the Edit tab on the upper left hand bar: Edit > Click on Notebook Settings > Enable GPU

In [3]:
import os
# Make a directory in your google drive namde "MICCAI_2020_reg_tutorial"
if not os.path.exists("./MICCAI_2020_reg_tutorial"):
  os.makedirs("./MICCAI_2020_reg_tutorial")
# Move into the dir
%cd ./MICCAI_2020_reg_tutorial
! pwd

/content/MICCAI_2020_reg_tutorial
/content/MICCAI_2020_reg_tutorial


In [4]:
%%capture
# Clone the DeepReg repository which contains the code
! git clone https://github.com/DeepRegNet/DeepReg.git
%cd ./DeepReg/
# pip install into the notebook env
! pip install -e .
# ! pip install wget
! pip install pyyaml==5.1
! pwd

# Introduction to Registration <a name="IntroReg"></a>

Image registration is an essential process in many clinical applications and computer assisted interventions **cite**. 

Applications of medical image registration (MIR) include - but are not limited to:
* Longitudinal comparison of images for a given patient with the same imaging modality: for example, comparing the outcome of given cancer treatment in a patients' CT scans over time. **cite**
* Inter-subject comparison: for example, population study using neuroimaging. **cite**
* Multi-modal registration for image guided surgery: for example, aligning real-time ultrasound scans to pre-operative CT or MRI scans to real-time achieve guidance in neurosurgical or abdominal applications. **cite**
* Atlas-based image segmentation: aligning new images to those carefully segmented, such that the reference segmentations can be propagated to those new images.

Image registration is the mapping of one image coordinate system to another, and can be sub-divided into rigid registrations and non-rigid registrations, depending on whether or not higher-dimesional tissue deformations is modelled as oppose to, for exmaple, a 6 degree-of-freedom (3 translational axes + 3 rotational axes) srigid transformation. Data may be aligned in many ways - spatially or temporally being two key ones.

Typically, we refer to one of the images in the pair as the *moving* image and the other as the *fixed* image. The goal is to find the *correspondence* that aligns the moving image to the fixed image - the transform will project the *moving* coordinates into the *fixed* coordinate space. The correspondence specifies the mapping between all voxels from one image to those from another image. The correspondence can be represented by a dense displacment field (DDF), defined as a set of displacement vectors for all pixels or voxels from one image to another. By using these displacement vectors, one image can be "warped" to become more "similar" to another.


## Classical Registration Methods
Image registration has been an active area of research for decades. Historically, the so-called classical algorithms pose registration as an optimisation problem between a given pair of moving and fixed images. In these methods, a pre-defined transformation model, rigid or nonrigid, is iteratively optimised to minimise a similarty measure - a metric that quantifies how "similar" between the warped moving image and the fixed image. 

The similarity measure can be a function sampling only important image features (extracted from a pre-processing step) or directly sampling all intensity values from both images.

* **Feature-based registration**: for example, a transformation between point clouds - a type of features widely used in many applications - can be estimated using Iterative Closest Point (ICP) or coherent point drift (CPD), for rigid or nonrigid transformation, respectively. The basis of ICP is to iteratively minimise the distance between the two point clouds by matching the points from one set to the closest point in the other set. This is done by searching for the minimum Euclidean distance for each point to the other cloud(otherwise known as the L2 distance), simply, $$L_2 = \sqrt(\vec{x_1} - \vec{x_2})^2$$ The transformation can be estimated from the found set of pairs, or correspondence, applying the transformation, and repeating the process many times.

  Another type of features are additional labels to highlight specific regions of interest (ROIs) to the clinician in the scan(s) - *segmentations*. These are typically provided as binary masks of organs that match the image dimensions, and refer to specific images or sets of images. While these data can be used to compute other types of similarity measures (such as overlap measure Dice) to drive a registration algorithm, they may be considered as weak labels to drive a learning-based registration algotithms - therefore we refere these segmentations as labels in this tutorial - more on this later. ;)

* **Intensity-based registration**: Typically, medical imaging data does not come in point cloud format, but rather, 2D, 3D and 4D matrices with a range of intensity values at each pixel or voxel. As such, different measures can be used directly on the intensity distributions of the data to measure similarity between the data. Examples of measures are cross correlation, mutual information and simple sum-square-difference - these intensity-based algorithms can optimise a transformation model directly using images without the feature extraction step.

Many today's deep-learning-based methods have been heavily influnced and derived from these prior work. (You can cite Dave's book here)


## Why use Deep Learning for MIR?
Usually, classical methods are unable to handle real-time registration of large point sets or feature intensity distributions owing to their computationally intense nature, especially in the case of 3D nonrigid registration. State-of-the-art classical methods are implemented on GPU still struggle for real-time performance for many time-critical clinical applications.

Secondly, classical algorithms are inherently pairwise approaches that can not take into account population data statistics directly and relying on well-designed transformation models and valid similarity being avaialable and robust, challenging for many real-world tasks.

In contrast, the computationally efficient inference and the ability to model complex, non-linear transformations of learning-based methods has motivated the development of neural networks which infer the optimal transformation from unseen data **cite**. 

However, it is important to point out that 
* Many deep-learning-based methods are still subject to these limitations, especially those that borrow transformation models and similarity measures directly from the classical methods
* Deep learning models are limited at inference time by how the model was trained - it is well known that deep learning models can overfit to the training data.
* Classical algorithms have been refined for many clinical applications and still work really well.

# Image Registration with Deep Learning <a name="DeepRegistrationIntro"></a>

In recent years, image registration has been re-formulated as a learning problem, in which, pairs of moving and fixed images are passed to a machine learning model (usually a neural network nowadays) to predict a transformation between the images.

In this tutorial, we investigate three factors that determine a deep learning approach for image registration:

1. What type of transformation is one trying to predict?
2. What type image data are being registered? Are there any other data, such as segmentations, to support the registration?
3. Are the data paired? Are they labeled?


## Types of transformations

We need to choose what type of transformation we want to predict. 

- **Predicting a dense displacement field**

  Given a pair of moving and fixed images, a registration network
  can be trained to output dense displacement field (DDF) of the same shape as the moving image. Each value in the DDF
  can be considered as the placement of the corresponding pixel / voxel of the moving
  image. Therefore, the DDF defines a mapping from the moving image's coordinates to the
  fixed image.

  In this tutorial, we mainly focus on DDF-based methods.

- **Predict a dense velocity field**

  Another option is to predict a dense velocity field (DVF) between a pair of images, such that a diffeomorphic
  DDF can be numerically integrated. Read
  ["A fast diffeomorphic image registration algorithm"](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.474.1033&rep=rep1&type=pdf)
  for more details.

- **Predict an affine transformation**

  A more constrained option is to predict an affine transformation, parameterised the
  affine transformation matrix of 12 degrees of freedom. The DDF can then be computed to
  resample the moving images in fixed image space.

- **Predict a region of interest**

  Instead of outputting the transformation between coordinates, given moving image,
  fixed image, and a region of interest (ROI) in the moving image, the network can
  predict the ROI in fixed image directly. Interested readers are referred to the MICCAI
  2019 paper:
  [Conditional segmentation in lieu of image registration](https://arxiv.org/abs/1907.00438).


## Data availability, level of supervision and network training strategies

Depending on the availability of the data labels, registration networks can be trained
with different approaches. These will influence our loss choice.

### Unsupervised

When the data label is unavailable, training can be achieved via an unsupervised loss.
The
following is an illustration of an unsupervised DDF-based registration network.

![Unsupervised DDF-based registration network](https://deepregnet.github.io/DeepReg/asset/registration-ddf-nn-unsupervised.svg)

The loss function often consists of the intensity based loss and deformation loss.

### Weakly-supervised

When there is no intensity based loss that is appropriate for the image pair one would
like to register, the training can take a pair of corresponding moving and fixed labels
(in addition to the image pair), represented by binary masks, to compute a label
dissimilarity (feature based loss) to drive the registration.

Combined with the regularisation on the predicted displacement field, this forms a
weakly-supervised training. An illustration of an weakly-supervised DDF-based
registration network is provided below.

When multiple labels are available for each image, the labels can be sampled during
training iteration, such that only one label per image is used in each iteration of the
data set (epoch). Read [data sampling API](tutorial_sampling.md) for more details.

![Weakly-supervised DDF-based registration network](https://deepregnet.github.io/DeepReg/asset/registration-ddf-nn-weakly-supervised.svg)

### Combined

When the data label is available, combining intensity based, feature based, and
deformation based losses together has shown superior registration accuracy, compared to
unsupervised and weakly supervised methods. Following is an illustration of a combined
DDF-based registration network.

![Combined DDF-based registration network](https://deepregnet.github.io/DeepReg/asset/registration-ddf-nn-combined.svg)

## Loss functions

We aim to train a network to predict some transformation between a pair of images that is likely. To do this, we need to define what is a "likely" transformation. This is done via a *loss function*.

The loss function defined to train a registration network will depend on the type of data we have access to.

- **Intensity based (image based) loss**

  This type of loss measures the dissimilarity of the fixed image and warped moving
  image, which is adapted from the classical image registration methods. Intensity based
  loss is modality-independent and similar to many other well-studied computer vision
  and medical imaging tasks, such as image segmentation.

  The common loss functions are normalized cross correlation (NCC), sum of squared
  distance (SSD), and normalized mutual information (MI).

- **Feature based (label based) loss**

  Provided labels for the input images, a feature based loss may be used to measure the (dis)similarity of warped regions of interest. Having computed a transformation between images using the net, one of the labels is warped and compared to the ground truth image label.
 Labels are typically manually contoured organs.

  The common loss function is Dice loss, Jacard and average cross-entropy over all
  voxels, which are measures of the overlap of the ROIs. The loss will minimise the negative overlap measure (eg. l = 1 - dice_score) to maximise overlap of the regions during training.

- **Deformation loss**

  Additionally, training may be regularised by computing the "likelihood" of a given displacement field. High deformation losses point to very unlikely displacement due to high gradients of the field - typically, deformation losses ensure smoothness in the displacement field. For DDFs, typical regularisation losses are bending energy losses, L1 or L2 norms of the displacement gradients.


## Image Registration with Deep Learning: Summary <a name="DeepRegistrationIntro"></a>

For deep learning methods, pairs of images, denoted as moving
and fixed images, are passed to the network to predict a transformation between the images.

The deep learning approach for MIR will depend on mainly three factors:

1. What type of transformation is one trying to predict?
2. What type image data are being registered? Are there any other data, such as segmentations, to support the registration?
3. Are the data paired? Are they labeled?

From this, we can design an appropriate architecture and choose an adequate loss function to motivate training.

# Classical Registration Examples <a name="classical-examples"></a>

We will define "single-layer" examples using functions from DeepReg, and train it to register two images.

First, we will illustrate the possibility of "self-registering" an image to it's affine-transformed counterpart, using head and neck CT scans data.

Then, we will nonrigid-register inter-subject scans, using MR images from two prostate cancer patients.

## Learning an affine transformation: a "self-registration"

In [ ]:
# We import some utility modules.
import nibabel
import tensorflow as tf 
import deepreg.model.layer as layer
import deepreg.model.loss.image as image_loss
import deepreg.model.loss.deform as deform_loss
import deepreg.model.layer_util as layer_util
import matplotlib.pyplot as plt
import os
import h5py
from tensorflow.keras.utils import get_file

# We set the plot size to some parameters.
plt.rcParams["figure.figsize"] = (100,100)

In [ ]:
# We define some utility functions first
## optimisation
@tf.function
def train_step_CT(grid, weights, optimizer, mov, fix):
    """
    Train step function for backprop using gradient tape
    :param grid: reference grid return from util.get_reference_grid
    :param weights: trainable affine parameters [1, 4, 3]
    :param optimizer: tf.optimizers
    :param mov: moving image [1, m_dim1, m_dim2, m_dim3]
    :param fix: fixed image [1, f_dim1, f_dim2, f_dim3]
    :return loss: image dissimilarity to minimise
    """
    with tf.GradientTape() as tape:
        pred = layer_util.resample(vol=mov, loc=layer_util.warp_grid(grid, weights))
        loss = image_loss.dissimilarity_fn(
            y_true=fix, y_pred=pred, name=image_loss_name
        )
    gradients = tape.gradient(loss, [weights])
    optimizer.apply_gradients(zip(gradients, [weights]))
    return loss

def plot_results(moving_image, fixed_image, warped_moving_image, nIdx):
  """
  Plotting the results from training
  :param moving_image: [IM_SIZE_0, IM_SIZE_1, 3]
  :param fixed_image:  [IM_SIZE_0, IM_SIZE_1, 3]
  :param warped_moving_image:  [IM_SIZE_0, IM_SIZE_1, 3]
  :param nIdx: number of indices to display
  """
  # Display
  plt.figure()
  for idx in range(nIdx):
      axs = plt.subplot(nIdx, 3, 3 * idx + 1)
      axs.imshow(moving_image[0, ..., idx_slices[idx]], cmap="gray")
      axs.axis("off")
      axs = plt.subplot(nIdx, 3, 3 * idx + 2)
      axs.imshow(fixed_image[0, ..., idx_slices[idx]], cmap="gray")
      axs.axis("off")
      axs = plt.subplot(nIdx, 3, 3 * idx + 3)
      axs.imshow(warped_moving_image[0, ..., idx_slices[idx]], cmap="gray")
      axs.axis("off")
  plt.ion()
  plt.suptitle('Moving Image - Fixed Image - Warped Moving Image', fontsize=200)
  plt.show()

def display(moving_image, fixed_image):
  """
  Displaying our two image tensors to register
  :param moving_image: [IM_SIZE_0, IM_SIZE_1, 3]
  :param fixed_image:  [IM_SIZE_0, IM_SIZE_1, 3]
  """
  # Display
  idx_slices = [int(5+x*5) for x in range(int(fixed_image_size[3]/5)-1)]
  nIdx = len(idx_slices)
  plt.figure()
  for idx in range(len(idx_slices)):
      axs = plt.subplot(nIdx, 2, 2*idx+1)
      axs.imshow(moving_image[0,...,idx_slices[idx]], cmap='gray')
      axs.axis('off')
      axs = plt.subplot(nIdx, 2, 2*idx+2)
      axs.imshow(fixed_image[0,...,idx_slices[idx]], cmap='gray')
      axs.axis('off')
  plt.suptitle('Moving Image - Fixed Image', fontsize=200)
  plt.show()

In [ ]:
# We download the data first
MAIN_PATH = os.getcwd()
PROJECT_DIR = os.path.join(MAIN_PATH, "demos/classical_ct_headandneck_affine")

DATA_PATH = "dataset"
FILE_PATH = os.path.abspath(os.path.join(PROJECT_DIR, DATA_PATH, "demo.h5"))
ORIGIN = "https://github.com/YipengHu/example-data/raw/master/hnct/demo.h5"

if os.path.exists(FILE_PATH):
    os.remove(FILE_PATH)
else:
  os.makedirs(os.path.join(PROJECT_DIR, DATA_PATH))
get_file(FILE_PATH, ORIGIN)
print("CT head-and-neck data downloaded: %s." % FILE_PATH)


In [ ]:
## registration parameters
image_loss_name = "ssd"
learning_rate = 0.01
total_iter = int(1000)


# Opening the file
fid = h5py.File(FILE_PATH, "r")
fixed_image = tf.cast(tf.expand_dims(fid["image"], axis=0), dtype=tf.float32)

# normalisation to [0,1]
fixed_image = (fixed_image - tf.reduce_min(fixed_image)) / (
    tf.reduce_max(fixed_image) - tf.reduce_min(fixed_image)
)  

# generate a radomly-affine-transformed moving image using DeepReg utils
fixed_image_size = fixed_image.shape
transform_random = layer_util.random_transform_generator(batch_size=1, scale=0.2)

grid_ref = layer_util.get_reference_grid(grid_size=fixed_image_size[1:4])

grid_random = layer_util.warp_grid(grid_ref, transform_random)
moving_image = layer_util.resample(vol=fixed_image, loc=grid_random)

# warp the labels to get ground-truth using the same random affine, for validation
fixed_labels = tf.cast(tf.expand_dims(fid["label"], axis=0), dtype=tf.float32)
moving_labels = tf.stack(
    [
        layer_util.resample(vol=fixed_labels[..., idx], loc=grid_random)
        for idx in range(fixed_labels.shape[4])
    ],
    axis=4,
)


# affine transformation as trainable weights
var_affine = tf.Variable(
    initial_value=[
        [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [0.0, 0.0, 0.0]]
    ],
    trainable=True,
)

# Optimising
optimiser = tf.optimizers.Adam(learning_rate)
for step in range(total_iter):
    loss_opt = train_step_CT(grid_ref, var_affine, optimiser, moving_image, fixed_image)
    if (step % 50) == 0:  # print info
        tf.print("Step", step, image_loss_name, loss_opt)

Once the optimisation converges, we can use the optimised affine transformation to warp the moving images.

In [ ]:
## warp the moving image using the optimised affine transformation
grid_opt = layer_util.warp_grid(grid_ref, var_affine)
warped_moving_image = layer_util.resample(vol=moving_image, loc=grid_opt)

idx_slices = [int(5 + x * 5) for x in range(int(fixed_image_size[3] / 5) - 1)]
nIdx = len(idx_slices)
# display
plot_results(moving_image, fixed_image, warped_moving_image, nIdx)

We can see especially from the first two slices how the data has registered to the fixed image. Let's see how the transformation appears on the labels.

In [ ]:
# Check how the labels have been registered
warped_moving_labels  = layer_util.resample(vol=moving_labels, loc=grid_opt)

# display
for idx_label in range(fixed_labels.shape[4]):
    plt.figure()
    for idx in range(len(idx_slices)):
        axs = plt.subplot(nIdx, 3, 3 * idx + 1)
        axs.imshow(moving_labels[0, ..., idx_slices[idx], idx_label], cmap="gray")
        axs.axis("off")
        axs = plt.subplot(nIdx, 3, 3 * idx + 2)
        axs.imshow(fixed_labels[0, ..., idx_slices[idx], idx_label], cmap="gray")
        axs.axis("off")
        axs = plt.subplot(nIdx, 3, 3 * idx + 3)
        axs.imshow(
            warped_moving_labels[0, ..., idx_slices[idx], idx_label], cmap="gray"
        )
        axs.axis("off")
    plt.ion()
    plt.show()

Here we can see how in some instances there is a label in the warped moving image versus the original moving image. This is an instance of conditional segmentation.

## Learning a nonrigid transformation: an inter-subject registration

In [ ]:
# Defining some utility functions
@tf.function
def train_step(warper, weights, optimizer, mov, fix):
    """
    Train step function for backpropagation using gradient tape

    :param warper: warping function returned from layer.Warping
    :param weights: trainable ddf [1, f_dim1, f_dim2, f_dim3, 3]
    :param optimizer: tf.optimizers
    :param mov: moving image [1, m_dim1, m_dim2, m_dim3]
    :param fix: fixed image [1, f_dim1, f_dim2, f_dim3]
    :return:
        loss: overall loss to optimise
        loss_image: image dissimilarity
        loss_deform: deformation regularisation
    """
    with tf.GradientTape() as tape:
        pred = warper(inputs=[weights, mov])
        # Calculating the image loss between the ground truth and prediction
        loss_image = image_loss.dissimilarity_fn(
            y_true=fix, y_pred=pred, name=image_loss_name
        )
        # We calculate the deformation loss
        loss_deform = deform_loss.local_displacement_energy(weights, deform_loss_name)
        # Total loss is weighted
        loss = loss_image + weight_deform_loss * loss_deform
    # We calculate the gradients by backpropagating the loss to the trainable layer,
    # which for registration is our ddf
    gradients = tape.gradient(loss, [weights])
    # Using our tf optimizer, we apply the gradients
    optimizer.apply_gradients(zip(gradients, [weights]))
    return loss, loss_image, loss_deform

In [ ]:
## We download the data for this example.
MAIN_PATH = os.getcwd()

DATA_PATH = "dataset"
if not os.path.exists(os.path.join(MAIN_PATH, DATA_PATH)):
  os.makedirs(os.path.join(MAIN_PATH, DATA_PATH))

FILE_PATH = os.path.abspath(os.path.join(MAIN_PATH, DATA_PATH, "demo2.h5"))
ORIGIN = "https://github.com/YipengHu/example-data/raw/master/promise12/demo2.h5"

get_file(FILE_PATH, ORIGIN)
print("Prostate MR data downloaded: %s." % FILE_PATH)

os.chdir(MAIN_PATH)

DATA_PATH = "dataset"
FILE_PATH = os.path.join(MAIN_PATH, DATA_PATH, "demo2.h5")

fid = h5py.File(FILE_PATH, "r")

In [ ]:
## We define some registration parameters
image_loss_name = "lncc" # local normalised cross correlation loss between images
deform_loss_name = "bending" # Loss to measure the bending energy of the ddf
weight_deform_loss = 10 # we weight the deformation loss
learning_rate = 0.1
total_iter = int(3000) # This will train for longer

In [ ]:
# We get our two subject images from our datasets
moving_image = tf.cast(tf.expand_dims(fid["image0"], axis=0), dtype=tf.float32)
fixed_image = tf.cast(tf.expand_dims(fid["image1"], axis=0), dtype=tf.float32)

In [ ]:
# We initialise our layers
fixed_image_size = fixed_image.shape

# Creating our DDF tensor that can be trained
# The DDF will be of shape [IM_SIZE_1, IM_SIZE_2, 3],
# representing the displacement field at each dimension.
var_ddf = tf.Variable(initialiser(fixed_image_size + [3]), name="ddf", trainable=True)

# We create a warping layer and initialise an optimizer
warping = layer.Warping(fixed_image_size=fixed_image_size[1:4])
optimiser = tf.optimizers.Adam(learning_rate)


## Training the layer
## With GPU this takes about 5 minutes.
for step in range(total_iter):
    # Call the gradient tape function
    loss_opt, loss_image_opt, loss_deform_opt = train_step(
        warping, var_ddf, optimiser, moving_image, fixed_image
    )
    if (step % 50) == 0:  # print info at every 50th step
        tf.print(
            "Step",
            step,
            "loss",
            loss_opt,
            image_loss_name,
            loss_image_opt,
            deform_loss_name,
            loss_deform_opt,
        )
        # Visualising loss during training
        # plt.figure()
        # fig, axs = plt.subplots(1, 3)
        # warped_moving_image = warping(inputs=[var_ddf, moving_image])
        # axs[0].imshow(moving_image[0, ..., 12])
        # axs[1].imshow(fixed_image[0, ..., 12])
        # axs[2].imshow(warped_moving_image[0, ..., 12])
        # plt.show()

In [ ]:
## Warp the moving image using the optimised ddf and the warping layer.
idx_slices = [int(5 + x * 5) for x in range(int(fixed_image_size[3] / 5) - 1)]
nIdx = len(idx_slices)
warped_moving_image = warping(inputs=[var_ddf, moving_image])
plot_results(moving_image, fixed_image, warped_moving_image, nIdx)

In [ ]:
## We can observe the effects of the warping on the moving label using the optimised affine transformation
moving_label = tf.cast(tf.expand_dims(fid["label0"], axis=0), dtype=tf.float32)
fixed_label = tf.cast(tf.expand_dims(fid["label1"], axis=0), dtype=tf.float32)

idx_slices = [int(5 + x * 5) for x in range(int(fixed_image_size[3] / 5) - 1)]
nIdx = len(idx_slices)
warped_moving_labels = warping(inputs=[var_ddf, moving_label])
plot_results(moving_label, fixed_label, warped_moving_labels, nIdx)

# Medical Image registration using an adapted DeepReg Demo <a name="deep-example"></a>

Now, we will build a more complex demo to investigate a clinical case, using deep-learning.

We will perform an inter-subject registration: aligning ct images from different patients. The images are all acquired at the similar timepoint in the breathing cycle for all patients. This kind of registration is useful for determining how one stimulus affects multiple patietns. If a drug or invasive procedure is administered to multiple patients, registering the images from different patients can give medical professsionals a sense of how each patient is responding in comparison to others. An example of such an application can be seen in **[2]**.

The data files used in this tutorial have been pre-arranged in a folder, required by the DeepReg unpaired dataset loader, and can be downloaded as follows.

In [5]:
from tensorflow.keras.utils import get_file
import zipfile
import shutil
import os

MAIN_PATH = os.getcwd()
PROJECT_DIR = os.path.join(MAIN_PATH, "demos/unpaired_ct_lung/")
if os.path.exists(os.path.join(PROJECT_DIR,'data')):
  shutil.rmtree(os.path.join(PROJECT_DIR,'data'))

URL_ZIP = "https://github.com/YipengHu/example-data/archive/unpaired_ct_lung.zip"
data_zip = get_file(os.path.abspath(os.path.join(PROJECT_DIR,'data.zip')), URL_ZIP)
with zipfile.ZipFile(data_zip, "r") as zf:
    zf.extractall(PROJECT_DIR)

DEF_PATH = os.path.join(PROJECT_DIR,"example-data-unpaired_ct_lung")
os.rename(DEF_PATH, os.path.join(PROJECT_DIR,'data'))

if os.path.exists(data_zip):
    os.remove(data_zip)

print("Files downloaded and unzipped.")

140484608/Unknown - 13s 0us/stepFiles downloaded and unzipped.


In [2]:

from deepreg.train import train
! ls
! pwd

"""
import tensorflow as tf
print(tf.test.gpu_device_name())
"""
import os

######## NOW WE DO THE TRAINING ########

MAIN_PATH = os.getcwd()

gpu = "0"
gpu_allow_growth = False
ckpt_path = ""
log_dir = "learn2reg_t2_unpaired_train_logs"

config_path = [
    os.path.join(MAIN_PATH, "deepreg/config/test/ddf.yaml"),
     os.path.join(MAIN_PATH, "demos/unpaired_ct_lung/unpaired_ct_lung.yaml")
]

# We use
train(
    gpu=gpu,
    config_path=config_path,
    gpu_allow_growth=gpu_allow_growth,
    ckpt_path=ckpt_path,
    log_dir=log_dir,
)

MICCAI_2020_reg_tutorial  sample_data
/content


FileNotFoundError: ignored

In [ ]:

from deepreg.predict import predict

######## PREDICTION ########

log_dir = "learn2reg_t2_unpaired_train_logs"
ckpt_path = os.path.join("logs", log_dir, "save", "weights-epoch2.ckpt")

gpu = ""
gpu_allow_growth = False
predict(
    gpu=gpu,
    gpu_allow_growth=gpu_allow_growth,
    ckpt_path=ckpt_path,
    mode="test",
    batch_size=1,
    log_dir=log_dir,
    sample_label="all",
)

# the numerical metrics are saved in the logs directory specified

######## VISUALISATION ########

# Now lets load in a few samples from the predicitons and plot them

# change the following line to the path to image0 label0
path_to_image0_label0 = r""
os.chdir(path_to_image0_label0)

# change image names if different images need to be plotted instead

plt.subplot(3, 2, 1)
label144 = plt.imread("depth144_fixed_label.png")
plt.imshow(label144)
plt.title("Label")
plt.axis("off")

plt.subplot(3, 2, 2)
pred144 = plt.imread("depth144_fixed_pred.png")
plt.imshow(pred144)
plt.title("Prediction")
plt.axis("off")


plt.subplot(3, 2, 3)
label145 = plt.imread("depth145_fixed_label.png")
plt.imshow(label145)
plt.axis("off")

plt.subplot(3, 2, 4)
pred145 = plt.imread("depth145_fixed_pred.png")
plt.imshow(pred145)
plt.axis("off")


plt.subplot(3, 2, 5)
label184 = plt.imread("depth184_fixed_label.png")
plt.imshow(label184)
plt.axis("off")

plt.subplot(3, 2, 6)
pred184 = plt.imread("depth184_fixed_pred.png")
plt.imshow(pred184)
plt.axis("off")

# this is the path where you want to save the visualisation as a png
path_to_save_fig = r""
plt.savefig(os.path.join(path_to_save_fig, "labels_and_preds.png"))

print("Visual representation of predictions saved to path specified")